# Example queries run on COVID-19 Knowledge Graph on Server
[Work in progress]

This notebook demonstrates how to run Cypher queries in a Jupyter Notebook by connecting to a database server.

In [1]:
import os
import time
import pandas as pd
from py2neo import Graph

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
graph = Graph("bolt://132.249.238.185:7687", user="reader", password="demo")

### List Node Metadata

In [4]:
query = """
MATCH (n:NodeMetadata)
RETURN n.name, n.shortDescription, n.description, n.example, n.details
"""
graph.run(query).to_data_frame()

,n.name,n.shortDescription,n.description,n.example,n.details
0,Location,Geographic location,A geograpic location,"World, ..., Country, State, Country, City, Cru...",None
1,World,The World,Top level location,None,None
2,UNRegion,Continental regions,Continental regions according to the M49 stan...,Americas,https://unstats.un.org/unsd/methodology/m49/
3,UNSubRegion,Subcontinental regions,Subcontinental regions according to the M49 st...,Latin America and the Caribbean,https://unstats.un.org/unsd/methodology/m49/
4,UNIntermediateRegion,Subdivisions of subcontinental regions,Subdivisions of subcontinental regions accordi...,Caribbean,https://unstats.un.org/unsd/methodology/m49/
5,Country,Countries and dependent Territories,Countries and dependent Territories defined b...,United States,http://www.geonames.org/
6,Admin1,"State, Province, Municipality","First administrative divisions, e.g, State, Pr...",California,http://www.geonames.org/
7,Admin2,County,Second administrative divisions: County in the US,San Diego County,http://www.geonames.org/
8,City,City,City,San Diego,http://www.geonames.org/
9,CruiseShip,Cruise ship,Cruise ship,Diamond Princess,http://www.productontology.org/doc/Cruise_ship


### List Organisms in KG

In [5]:
query = """
MATCH (p:Organism)
RETURN p.name as name, p.scientificName as scientificName, p.id as taxonomy
"""
graph.run(query).to_data_frame()

,name,scientificName,taxonomy
0,SARS-CoV-2,None,taxonomy:2697049
1,MERS-CoV,None,taxonomy:1335626
2,SARS-CoV,None,taxonomy:694009
3,human,None,taxonomy:9606
4,intermediate horseshoe bat,None,taxonomy:59477
5,Malayan horseshoe bat,None,taxonomy:608659
6,horseshoe bat,None,taxonomy:49442
7,Malayan pangolin,None,taxonomy:9974
8,palm civet,None,taxonomy:71116
9,carnivores,None,taxonomy:9608


### List Coronavirus Outbreaks

In [7]:
query = """
MATCH (p:Pathogen)-[:CAUSES]->(o:Outbreak)
RETURN p.name as name, p.scientificName as scientificName, p.id as taxonomy, o.id as outbreak, o.startDate as startDate
"""
graph.run(query).to_data_frame()

,name,scientificName,taxonomy,outbreak,startDate
0,SARS-CoV-2,None,taxonomy:2697049,COVID-19,2019
1,MERS-CoV,None,taxonomy:1335626,MERS,2012
2,SARS-CoV,None,taxonomy:694009,SARS,2003


### List Strains that are mentioned in PubMed Central Articles

In [9]:
query = """
MATCH (p:Publication)-[:MENTIONS]->(s:Strain)<-[:CARRIES]-(h:Host)
RETURN p.id as pmc, s.name as name, s.collectionDate  as collectionDate, h.name as host, s.id as host_id
ORDER by s.collectionDate
"""
graph.run(query).to_data_frame().head(20)

,pmc,name,collectionDate,host,host_id
0,pmc:PMC7166773,TG13,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131
1,pmc:PMC7166309,TG13,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131
2,pmc:PMC7095418,TG13,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131
3,pmc:PMC7118693,TG13,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131
4,pmc:PMC7205519,TG13,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131
5,pmc:PMC7194065,TG13,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131
6,pmc:PMC7156227,TG13,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131
7,pmc:PMC7182198,TG13,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131
8,pmc:PMC7230814,TG13,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131
9,pmc:PMC7106073,TG13,2013-07-24,intermediate horseshoe bat,https://www.gisaid.org/EPI_ISL_402131


### List Gene and Protein information for Reference Genome
This query lists the genes and proteins encoded by the SARS-CoV-2 reference genome. This is the first genome of SARS-CoV-2 collected in Wuhan on Dec. 5, 2019.

In [10]:
query = """
MATCH (s:Strain)-[:HAS]->(g:Gene)-[:ENCODES]->(p:Protein)
RETURN s.id as referenceGenome, s.name as name, s.collectionDate  as collectionDate, 
       g.name as gene, g.id as geneId, p.name as protein, p.id as protein_id 
ORDER by s.collectionDate
"""
graph.run(query).to_data_frame()

,referenceGenome,name,collectionDate,gene,geneId,protein,protein_id
0,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-07,ORF1ab,ncbigene:43740578-266-21555,Replicase polyprotein 1ab,md5:e6608b50fcd6e004708a875615ddf2d9
1,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-07,ORF1ab,ncbigene:43740578-266-13483,Replicase polyprotein 1a,md5:e781b58591b8dbdd15f84dcbdec82105
2,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-07,S,ncbigene:43740568-21563-25384,Spike glycoprotein,md5:4c35f09aac2f7be4f3cffd30c6aecac8
3,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-07,ORF3a,ncbigene:43740569-25393-26220,Protein 3a,md5:f5c8b89ceac3f14e456577557df1ef40
4,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-07,E,ncbigene:43740570-26245-26472,Envelope small membrane protein,md5:375e0f905c315e06a99c80b736c125d2
5,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-07,M,ncbigene:43740571-26523-27191,Membrane protein,md5:1cd6abff79ad3633e17582eb0e576539
6,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-07,ORF6,ncbigene:43740572-27202-27387,Non-structural protein 6,md5:90b50e0be9abd893bd32b163d6933f0c
7,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-07,ORF7a,ncbigene:43740573-27394-27759,Protein 7a,md5:f65213344e2e68de1cae4feb9c5e07b1
8,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-07,ORF7b,ncbigene:43740574-27756-27887,Protein non-structural 7b,md5:c7f0179da4ca26456ee905081f485cc8
9,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-07,ORF8,ncbigene:43740577-27894-28259,Non-structural protein 8,md5:3a77ad9207d29beacfbf6f18b0da9e26


### Cases in a specific County (Admin2)

In [11]:
admin2 = 'San Diego County'

query = """
MATCH (c:Cases{date: date("2020-05-04")})-[:REPORTED_IN]->(a:Admin2{name: $admin2})
RETURN a.name as name, c.cummulativeConfirmed as confirmed, c.cummulativeDeaths as deaths
"""
graph.run(query, admin2=admin2).to_data_frame()

,name,confirmed,deaths
0,San Diego County,3927,139


### Aggregate cases by State (Admin1)

In [12]:
query = """
MATCH (o:Outbreak{id: "COVID-19"})<-[:RELATED_TO]-(c:Cases{date: date("2020-05-04")})-[:REPORTED_IN]->(a:Admin2)-[:IN]->(a1:Admin1)
RETURN a1.name as state, sum(c.cummulativeConfirmed) as confirmed, sum(c.cummulativeDeaths) as deaths
ORDER BY deaths DESC
"""
graph.run(query).to_data_frame()

,state,confirmed,deaths
0,New York,318953,24999
1,New Jersey,127622,7910
2,Massachusetts,68393,4082
3,Michigan,41611,4069
4,Pennsylvania,52816,2838
5,Illinois,63012,2625
6,Connecticut,29610,2553
7,California,54260,2247
8,Louisiana,25473,1636
9,Florida,36414,1370
